In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
fights_df = pd.read_csv('fights.csv')
fighters_df = pd.read_csv('fighters.csv')

FileNotFoundError: ignored

In [ ]:
# Remove fighters with no stats
num_fighters_removed = len(fighters_df)
fighters_df = fighters_df.drop(fighters_df[((fighters_df["slpm"] == 0) &
                               (fighters_df["str_acc"] ==0) &
                               (fighters_df["sapm"] == 0) &
                               (fighters_df["str_def"] == 0) &
                               (fighters_df["td_avg"] == 0) &
                               (fighters_df["td_acc"] == 0) &
                               (fighters_df["td_def"] == 0) &
                               (fighters_df["sub_avg"] == 0))].index)

fighters_df = fighters_df.dropna()

num_fighters_removed -= len(fighters_df)

print(f"{num_fighters_removed} fighters removed")

In [ ]:
# REMOVE FIGHTERS WITH SAME NAME
fighters_df[fighters_df.duplicated(subset="name", keep = False)]
fighters_df = fighters_df.drop(fighters_df[fighters_df["name"].isin(["Joey Gomez", "Bruno Silva"])].index)


# REMOVE FROM FIGHTS DATA ALSO
fights_df = fights_df.drop(fights_df[fights_df["fighter_1"].isin(["Joey Gomez", "Bruno Silva"]) | fights_df["fighter_2"].isin(["Joey Gomez", "Bruno Silva"])].index)


In [ ]:
# DROP RESULT COLUMNS
fights_df = fights_df.drop(["_id"], axis=1)
fighters_df.set_index("name", inplace=True)

In [ ]:
# Verify all fighters in fights table are in fighters table

all_fighter_names = fighters_df.index.values.tolist()

fights_df = fights_df.loc[(fights_df["fighter_1"].isin(all_fighter_names)) &
                              (fights_df["fighter_2"].isin(all_fighter_names))]



fights_df.reset_index(inplace=True, drop=True)

In [ ]:
fighter1_data = fighters_df.loc[fights_df["fighter_1"]]
fighter1_data = fighter1_data.add_suffix('_fighter_1')
fighter2_data = fighters_df.loc[fights_df["fighter_2"]]
fighter2_data = fighter2_data.add_suffix('_fighter_2')

fighter1_data.reset_index(inplace=True, drop=True)
fighter2_data.reset_index(inplace=True, drop=True)
fights_df = pd.concat([fights_df, fighter1_data, fighter2_data], axis=1, sort=False)

In [ ]:
# ADD WINNER COLUMN
fights_df["winner"] = 0
fights_df.loc[fights_df["result_fighter_2"]=="W","winner"]= 1

# DROP NC AND DRAWS
fights_df = fights_df.drop(fights_df[~fights_df["result_fighter_1"].isin(["W", "L"])].index)

# DROP RESULT COLUMNS
fights_df = fights_df.drop(["result_fighter_1", "result_fighter_2"], axis=1)

# COLUMNS NOT USED IN MODEL
fights_df = fights_df.drop([ "event", "title", "referee", 'ctrl_fighter_1',
 'ctrl_fighter_2'], axis=1)

# REMOVE FIGHTS WHICH ARENT IN CURRENT UFC FORMAT
fights_df = fights_df.drop(fights_df[~fights_df["time format"].isin(["5 Rnd (5-5-5-5-5)", "3 Rnd (5-5-5)"])].index)

fights_df.head()


In [ ]:
fights_df.to_csv("drive/MyDrive/ufc_data/v2/fights_final")